### The project has evolved. The CSV files are now in the olist.db database. For the project with the old schema between the CSV files, it might be useful to create a mapping between the old CSV files and the database tables, as well as the common keys between the tables. https://drive.google.com/file/d/1cC1h5ZiakQMM6Ut9Hqf13r-jTpMHV8d5/view?usp=sharing  

### Schema of liaison between data when data where csv files see https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

![data_schema.PNG](../data/data_schema.PNG)


Here is a mapping table in text format between the CSV files and the tables in `olist.db`, including the identical keys between tables when they exist:

| CSV File                               | SQLite Table       | Identical Keys                            |
|----------------------------------------|--------------------|-------------------------------------------|
| olist_customers_dataset.csv            | customers          | customer_id, customer_zip_code_prefix     |
| olist_geolocation_dataset.csv          | geoloc             | geolocation_zip_code_prefix               |
| olist_order_items_dataset.csv          | order_items        | order_id, product_id, seller_id           |
| olist_order_payments_dataset.csv       | order_pymts        | order_id                                  |
| olist_order_reviews_dataset.csv        | order_reviews      | order_id                                  |
| olist_orders_dataset.csv               | orders             | customer_id, order_id                     |
| olist_products_dataset.csv             | products           | product_id                                |
| olist_sellers_dataset.csv              | sellers            | seller_id, seller_zip_code_prefix         |
| product_category_name_translation.csv  | translation        | product_category_name                     |

### Mapping Details

1. **olist_customers_dataset.csv -> customers**
   - Identical Keys: `customer_id`, `customer_zip_code_prefix`

2. **olist_geolocation_dataset.csv -> geoloc**
   - Identical Keys: `geolocation_zip_code_prefix`

3. **olist_order_items_dataset.csv -> order_items**
   - Identical Keys: `order_id`, `product_id`, `seller_id`

4. **olist_order_payments_dataset.csv -> order_pymts**
   - Identical Keys: `order_id`

5. **olist_order_reviews_dataset.csv -> order_reviews**
   - Identical Keys: `order_id`

6. **olist_orders_dataset.csv -> orders**
   - Identical Keys: `customer_id`, `order_id`

7. **olist_products_dataset.csv -> products**
   - Identical Keys: `product_id`

8. **olist_sellers_dataset.csv -> sellers**
   - Identical Keys: `seller_id`, `seller_zip_code_prefix`

9. **product_category_name_translation.csv -> translation**
   - Identical Keys: `product_category_name`

### Explanation
- Identical keys are used to establish relationships between different tables, thus facilitating the necessary joins for complex queries.
- For example, `order_id` is a common key between the tables `orders`, `order_pymts`, `order_reviews`, and `order_items`, allowing the linking of order information, payment, reviews, and order items.

This mapping table can serve as a reference to understand how the data from the CSV files is structured in the `olist.db` database and how they can be joined to answer analytical questions.

### Olist E-Commerce connection to the database check

In [13]:
import pandas as pd
import sqlite3
import os

# Relative path to the database
db_path = os.path.join('..', 'data', 'olist.db')

if os.path.exists(db_path):
    print("Connecting to the database...")
    conn = sqlite3.connect(db_path)
    print("Database connected.")
    
    query = "SELECT COUNT(*) FROM orders;"
    try:
        data = pd.read_sql(query, conn)
        print("Total number of rows in orders table:", data.iloc[0, 0])
    except Exception as e:
        print("An error occurred:", e)
    finally:
        conn.close()
        print("Database connection closed.")
else:
    print("The file does not exist at the specified location:", db_path)



Connecting to the database...
Database connected.
Total number of rows in orders table: 99441
Database connection closed.


### Inspecting the Database Schema

In [14]:
import pandas as pd
import sqlite3
import os

# Relative path to the database
db_path = os.path.join('..', 'data', 'olist.db')

if os.path.exists(db_path):
    print("Connecting to the database...")
    
    # Create a connection to the database
    conn = sqlite3.connect(db_path)
    print("Database connected.")
    
    try:
        # List all tables
        tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
        tables = pd.read_sql(tables_query, conn)
        print("Tables in the database:")
        print(tables)

        # Inspect schema of the relevant tables
        for table in tables['name']:
            schema_query = f"PRAGMA table_info({table});"
            schema = pd.read_sql(schema_query, conn)
            print(f"Schema of {table}:")
            print(schema)
    except Exception as e:
        print("An error occurred:", e)
    finally:
        # Close the connection
        conn.close()
        print("Database connection closed.")
else:
    print("The file does not exist at the specified location:", db_path)



Connecting to the database...
Database connected.
Tables in the database:
            name
0      customers
1         geoloc
2    order_items
3    order_pymts
4  order_reviews
5         orders
6       products
7        sellers
8    translation
Schema of customers:
   cid                      name    type  notnull dflt_value  pk
0    0                     index  BIGINT        0       None   0
1    1               customer_id    TEXT        0       None   0
2    2        customer_unique_id    TEXT        0       None   0
3    3  customer_zip_code_prefix  BIGINT        0       None   0
4    4             customer_city    TEXT        0       None   0
5    5            customer_state    TEXT        0       None   0
Schema of geoloc:
   cid                         name    type  notnull dflt_value  pk
0    0                        index  BIGINT        0       None   0
1    1  geolocation_zip_code_prefix  BIGINT        0       None   0
2    2              geolocation_lat   FLOAT        0      

### Checking for Common Values in Key Columns

In [15]:
import pandas as pd
import sqlite3
import os

# Relative path to the database
db_path = os.path.join('..', 'data', 'olist.db')

# Function to execute SQL queries and return DataFrames
def execute_query(query):
    with sqlite3.connect(db_path) as conn:
        return pd.read_sql(query, conn)

# Function to get unique values of a column
def get_unique_column_values(table, column):
    query = f"SELECT DISTINCT {column} FROM {table}"
    df = execute_query(query)
    return set(df[column].values)

# List of tables and columns to check
tables_columns = {
    'customers': ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix'],
    'geoloc': ['geolocation_zip_code_prefix'],
    'order_items': ['order_id', 'product_id', 'seller_id'],
    'order_pymts': ['order_id'],
    'order_reviews': ['order_id', 'review_id'],
    'orders': ['order_id', 'customer_id'],
    'products': ['product_id'],
    'sellers': ['seller_id', 'seller_zip_code_prefix'],
    'translation': ['product_category_name'],
}

# Function to compare columns and check for common values
def compare_columns(tables_columns):
    common_values_found = False
    columns_to_compare = {}
    
    for table, columns in tables_columns.items():
        for column in columns:
            columns_to_compare[f"{table}.{column}"] = get_unique_column_values(table, column)
    
    keys = list(columns_to_compare.keys())
    
    for i in range(len(keys)):
        table_column1 = keys[i]
        values1 = columns_to_compare[table_column1]
        for j in range(i + 1, len(keys)):
            table_column2 = keys[j]
            values2 = columns_to_compare[table_column2]
            common_values = values1.intersection(values2)
            if common_values:
                print(f"Common values found between {table_column1} and {table_column2}:")
                print(f"Number of common values: {len(common_values)}")
                print(f"Sample common values: {list(common_values)[:5]}")  # Print only the first 5 common values as a sample
                common_values_found = True
    
    if not common_values_found:
        print("No common values found between the specified columns of different tables.")

# Specific validation for customer_id, order_id, and customer_unique_id
def compare_specific_columns(columns_to_compare):
    special_cases = [
        ('customers.customer_id', 'orders.order_id'),
        ('customers.customer_id', 'customers.customer_unique_id'),
        ('customers.customer_unique_id', 'orders.order_id')
    ]
    
    for case in special_cases:
        table_column1, table_column2 = case
        values1 = columns_to_compare.get(table_column1, set())
        values2 = columns_to_compare.get(table_column2, set())
        common_values = values1.intersection(values2)
        if common_values:
            print(f"Common values found between {table_column1} and {table_column2}: {common_values}")
        else:
            print(f"No common values found between {table_column1} and {table_column2}")    
            

# Include in main function
def compare_columns(tables_columns):
    common_values_found = False
    columns_to_compare = {}
    
    for table, columns in tables_columns.items():
        for column in columns:
            columns_to_compare[f"{table}.{column}"] = get_unique_column_values(table, column)
    
    keys = list(columns_to_compare.keys())
    
    for i in range(len(keys)):
        table_column1 = keys[i]
        values1 = columns_to_compare[table_column1]
        for j in range(i + 1, len(keys)):
            table_column2 = keys[j]
            values2 = columns_to_compare[table_column2]
            common_values = values1.intersection(values2)
            if common_values:
                print(f"Common values found between {table_column1} and {table_column2}:")
                print(f"Number of common values: {len(common_values)}")
                print(f"Sample common values: {list(common_values)[:5]}")  # Print only the first 5 common values as a sample
                common_values_found = True
    
    # Specific validation
    if compare_specific_columns(columns_to_compare):
        common_values_found = True
    
    if not common_values_found:
        print("No common values found between the specified columns of different tables.")

# Execute the column comparison
compare_columns(tables_columns)


Common values found between customers.customer_id and orders.customer_id:
Number of common values: 99441
Sample common values: ['9fe0cd1d9873dff9f76b9dd37009dc5a', '3ae0909507f3aff8f82173639d398da2', 'cf5c87f52f4f9d68b80bbaaea6afcb67', '0f684a3f14d10efbcb72b61453fc70d8', '30ff74ba6ed04c0069481436021bc4aa']
Common values found between customers.customer_zip_code_prefix and geoloc.geolocation_zip_code_prefix:
Number of common values: 14837
Sample common values: [65540, 65550, 65560, 98335, 98338]
Common values found between customers.customer_zip_code_prefix and sellers.seller_zip_code_prefix:
Number of common values: 2162
Sample common values: [81925, 81930, 8215, 8220, 8223]
Common values found between geoloc.geolocation_zip_code_prefix and sellers.seller_zip_code_prefix:
Number of common values: 2239
Sample common values: [81925, 81930, 8215, 8220, 8223]
Common values found between order_items.order_id and order_pymts.order_id:
Number of common values: 98665
Sample common values: ['8b

### Key Observations

1. **Expected Foreign Key Relationships**:
    - `order_items.order_id`, `order_pymts.order_id`, `order_reviews.order_id`, and `orders.order_id`: High numbers of common values, indicating expected foreign key relationships.
    - `order_items.product_id` and `products.product_id`: 32951 common values, indicating a correct relationship.
    - `order_items.seller_id` and `sellers.seller_id`: 3095 common values, indicating a correct relationship.

2. **Zip Code Relationships**:
    - `customers.customer_zip_code_prefix` and `geoloc.geolocation_zip_code_prefix`: 14837 common values.
    - `customers.customer_zip_code_prefix` and `sellers.seller_zip_code_prefix`: 2162 common values.
    - `geoloc.geolocation_zip_code_prefix` and `sellers.seller_zip_code_prefix`: 2239 common values.

### Specific Validation

1. **No Common Values**:
    - `customers.customer_id` and `orders.order_id`: No common values found.
    - `customers.customer_id` and `customers.customer_unique_id`: No common values found.
    - `customers.customer_unique_id` and `orders.order_id`: No common values found.

This indicates that the `customer_id` in the `customers` table is properly distinguished from `order_id` in the `orders` table and `customer_unique_id` in the `customers` table. This is the expected result and confirms that there are no unexpected overlaps between these IDs.

-  Common values found between customers.customer_id and orders.customer_id: Number of common values: 99441 => same values so

### Query to Display a Few Rows and the Date Format

In [16]:
import pandas as pd
import sqlite3
import os

# Relative path to the database
db_path = os.path.join('..', 'data', 'olist.db')

if os.path.exists(db_path):
    print("Connecting to the database...")
    conn = sqlite3.connect(db_path)
    print("Database connected.")
    
    query = """
    SELECT 
        order_id,
        customer_id,
        order_status,
        order_purchase_timestamp
    FROM orders
    LIMIT 5;
    """
    try:
        data = pd.read_sql(query, conn)
        print("Sample rows from orders table:")
        print(data)  # Display sample rows to check date format
    except Exception as e:
        print("An error occurred:", e)
    finally:
        conn.close()
        print("Database connection closed.")
else:
    print("The file does not exist at the specified location:", db_path)


Connecting to the database...
Database connected.
Sample rows from orders table:
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp  
0    delivered      2017-10-02 10:56:33  
1    delivered      2018-07-24 20:41:37  
2    delivered      2018-08-08 08:38:49  
3    delivered      2017-11-18 19:28:06  
4    delivered      2018-02-13 21:18:39  
Database connection closed.


### Query to Display the Current Date in order_purchase_timestamp Format

In [17]:
import pandas as pd
import sqlite3
import os

# Relative path to the database
db_path = os.path.join('..', 'data', 'olist.db')

if os.path.exists(db_path):
    print("Connecting to the database...")
    conn = sqlite3.connect(db_path)
    print("Database connected.")
    
    query = """
    SELECT strftime('%Y-%m-%d %H:%M:%S', 'now') AS current_date;
    """
    try:
        data = pd.read_sql(query, conn)
        print("Current date (formatted):")
        print(data)  # Display the current date to ensure it's being calculated correctly
    except Exception as e:
        print("An error occurred:", e)
    finally:
        conn.close()
        print("Database connection closed.")
else:
    print("The file does not exist at the specified location:", db_path)


Connecting to the database...
Database connected.
Current date (formatted):
          current_date
0  2024-07-24 16:56:19
Database connection closed.


### 1.Recent Orders with at Least 3 Days of Delay (Excluding Canceled Orders) for Orders Less Than 90 Days Old (latest date for order_purchase_timestamp is the latest date in the database)

### Explanation of the Query
1. **Recent Orders**: The query filters the orders based on the purchase timestamp to select only the recent orders placed in the last 90 days.
2. **Order Status**: It excludes the canceled orders by checking the order status.
3. **Delay Calculation**: The delay in delivery is calculated by finding the difference in days between the delivered customer date and the estimated delivery date.
4. **Common Table Expression (CTE)**: The query uses a Common Table Expression (CTE) named `RelevantOrders` to filter the orders based on the conditions specified.
5. **Result Columns**: The final result includes the `order_id`, `customer_id`, `order_status`, `order_purchase_timestamp`, `order_delivered_customer_date`, and `delay_days` columns for the selected orders.
6. **Data Loading**: The query result is loaded into a pandas DataFrame for further analysis and display.
7. **Display**: The first few rows of the resulting DataFrame are displayed to show the recent orders with at least 3 days of delay.
8. **Context**: The query considers the date range issue and provides additional information if no data is available due to older dates in the database.

In [18]:
import pandas as pd
import sqlite3
import os

# Relative path to the database
db_path = os.path.join('..', 'data', 'olist.db')

# Function to execute SQL queries and return results as a pandas DataFrame
def execute_query(query):
    with sqlite3.connect(db_path) as conn:
        return pd.read_sql(query, conn)

# Check if the file exists before attempting to connect
if os.path.exists(db_path):
    print("Connecting to the database...")
    
    try:
        # Create a connection to the database
        conn = sqlite3.connect(db_path)
        print("Database connected.")

        # Query Recent Orders with at Least 3 Days of Delay (Excluding Canceled Orders) for Orders Less Than 90 Days Old (latest date for order_purchase_timestamp is the latest date in the database)
        combined_query = """
        WITH LatestDate AS (
            SELECT MAX(order_purchase_timestamp) AS latest_order_date
            FROM orders
        ),
        RecentOrders AS (
            SELECT 
                order_id,
                customer_id,
                order_status,
                order_purchase_timestamp,
                order_delivered_customer_date,
                julianday(order_delivered_customer_date) - julianday(order_estimated_delivery_date) AS delay_days
            FROM orders, LatestDate
            WHERE 
                order_status <> 'canceled' 
                AND order_purchase_timestamp >= datetime(LatestDate.latest_order_date, '-90 days')
        )
        SELECT
            (SELECT latest_order_date FROM LatestDate) AS latest_order_date,
            (SELECT COUNT(*) FROM RecentOrders) AS recent_orders_count,
            order_id,
            customer_id,
            order_status,
            order_purchase_timestamp,
            order_delivered_customer_date,
            delay_days
        FROM RecentOrders
        WHERE delay_days >= 3;
        """

        # Execute the combined query and load data into a DataFrame
        data = execute_query(combined_query)
        
        # Extract and print the latest order date
        latest_order_date = data['latest_order_date'][0]
        print(f"Latest order purchase date in the database: {latest_order_date}")
        
        # Extract and print the count of recent orders
        recent_orders_count = data['recent_orders_count'][0]
        print(f"Number of recent orders in the last 90 days: {recent_orders_count}")

        # Drop the extra columns used for display purposes
        data = data.drop(columns=['latest_order_date', 'recent_orders_count'])

        if recent_orders_count == 0 or data.empty:
            print("No data available for recent orders with at least 3 days of delay (excluding canceled orders) for orders less than 90 days old.")
        else:
            print("Query executed successfully.")
            print(data.shape)  # Display the shape of the result
            print(data.head())  # Display the first few rows of the result
            
    except Exception as e:
        print("An error occurred:", e)
    finally:
        # Close the connection
        conn.close()
        print("Database connection closed.")
else:
    print("The file does not exist at the specified location:", db_path)


Connecting to the database...
Database connected.
Latest order purchase date in the database: 2018-10-17 17:30:18
Number of recent orders in the last 90 days: 9405
Query executed successfully.
(307, 6)
                           order_id                       customer_id  \
0  cfa4fa27b417971e86d8127cb688712f  7093250e1741ebbed41f0cc552025fd6   
1  234c056c50619f48da64f731c48242b4  44e460a655f7154ccd9faa4dbbbaf68a   
2  7f579e203c931f3e8410103359c6d523  d665be250d1c687c58fdea61a9b55a58   
3  cb6e441ff2ef574ce08d3709426f88ec  4fb843d304c57182d4aa27bb39ca592b   
4  03720fdc92032ee4abd471d172006ab0  116458665bac0ff47d5e87f65e8ec681   

  order_status order_purchase_timestamp order_delivered_customer_date  \
0    delivered      2018-08-16 09:44:23           2018-08-29 01:41:41   
1    delivered      2018-08-14 14:49:15           2018-09-01 18:14:42   
2    delivered      2018-08-02 18:09:27           2018-08-13 20:11:47   
3    delivered      2018-08-08 19:27:03           2018-08-18 01:11:

### 2.Sellers generating revenue over 100,000 Real via Olist (without considering the date range issue for the latest date for order_status as the latest date in the database)

### Explanation of the Query
1. **Revenue Calculation**: The query calculates the total revenue generated by each seller by summing the prices of the products they sold. It uses the `order_items` table to get the price of each product and the `orders` table to filter only the orders that are delivered (`order_status = 'delivered'`).
2. **Common Key**: The common key between the `order_items` and `sellers` tables is the `seller_id`, which is used to join the two tables and retrieve additional information about the sellers.
3. **Filtering Criteria**: The query filters the sellers based on the total revenue generated, selecting only those sellers who have generated over 100,000 Real.
4. **Result Columns**: The final result includes the `seller_id`, `seller_zip_code_prefix`, and `total_revenue` columns for the selected sellers.
5. **Data Loading**: The query result is loaded into a pandas DataFrame for further analysis and display.
6. **Display**: The first few rows of the resulting DataFrame are displayed to show the sellers who have generated revenue over 100,000 Real.
7. **Context**: The query considers the date range issue and provides additional information if no data is available due to older dates in the database.

In [19]:
import pandas as pd
import sqlite3
import os

# Relative path to the database
db_path = os.path.join('..', 'data', 'olist.db')

# Function to execute SQL queries and return DataFrames
def execute_query(query):
    with sqlite3.connect(db_path) as conn:
        return pd.read_sql(query, conn)

# SQL query for sellers generating revenue over 100,000 Real
query_revenue_sellers = """
WITH Revenue AS (
    SELECT
        i.seller_id,
        SUM(i.price) AS total_revenue
    FROM order_items AS i
    JOIN orders AS o ON i.order_id = o.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY i.seller_id
)
SELECT s.seller_id, s.seller_zip_code_prefix, r.total_revenue
FROM Revenue AS r
JOIN sellers AS s ON r.seller_id = s.seller_id
WHERE r.total_revenue > 100000;
"""

# Execute the query and load data into a DataFrame
data_revenue_sellers = execute_query(query_revenue_sellers)
print("DataFrame: revenue_sellers")
print(data_revenue_sellers.shape)
print(data_revenue_sellers[["seller_id", "total_revenue"]])  # Display the first few rows of the result
print()


DataFrame: revenue_sellers
(17, 3)
                           seller_id  total_revenue
0   7e93a43ef30c4f03f38b393420bc753a      165981.49
1   7d13fca15225358621be4086e1eb0964      112436.18
2   955fee9216a65b617aa5c0531780ce60      131836.71
3   1f50f920176fa81dab994f9023523100      106655.71
4   fa1c13f2614d7b5c4749cbc52fecda94      190917.14
5   6560211a19b47992c3666cc44a7e94c0      120702.83
6   53243585a1d6dc2643021fd1853d8905      217940.44
7   7c67e1448b00f6e969d365cea6b010ab      186570.05
8   cc419e0650a3c5ba77189a1882b7556a      101090.96
9   da8622b14eb17ae2831f4ac5b9dab84a      159816.87
10  620c87c171fb2a6dd6e8bb4dec959fc6      112461.50
11  7a67c85e85bb2ce8582c35f2203ad736      139658.69
12  5dceca129747e92ff8ef7a997dc4f8ca      111126.73
13  4a3ca9315b744ce9f8e9374361493884      196882.12
14  4869f7a5dfa277a7dca6462dcf3b52b2      226987.93
15  1025f0e2d44d7041d6cf58b6550e0bfa      138208.56
16  46dc3b2cc0980fb8ec44634e21d2718e      122811.38



### 3. Who are the new sellers (less than 3 months of seniority) who are already highly engaged with the platform (having already sold more than 30 products) without considering the date range issue (latest date for order_purchase_timestamp as it is in the database)

### Explanation of the Query
1. **New Sellers**: The query identifies new sellers who have been active on the platform for less than 90 days. It calculates the seller's seniority based on the date of their first order.
2. **Highly Engaged Sellers**: The query filters the new sellers who have already sold more than 30 products, indicating high engagement with the platform.
3. **Common Table Expressions (CTEs)**: The query uses two Common Table Expressions (CTEs) to first find the first order date for each seller and then calculate the total number of products sold by each seller.
4. **Result Columns**: The final result includes the `seller_id` and `seller_zip_code_prefix` columns for the selected new sellers who are highly engaged with the platform.
5. **Data Loading**: The query result is loaded into a pandas DataFrame for further analysis and display.
6. **Display**: The first few rows of the resulting DataFrame are displayed to show the new sellers who are highly engaged with the platform.
7. **Context**: The query considers the date range issue and provides additional information if no data is available due to older dates in the database.

In [20]:
import pandas as pd
import sqlite3
import os
from datetime import datetime

# Get today's date
today = datetime.today().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD

# Relative path to the database
db_path = os.path.join('..', 'data', 'olist.db')

# Function to execute SQL queries and return DataFrames
def execute_query(query):
    with sqlite3.connect(db_path) as conn:
        return pd.read_sql(query, conn)

# SQL query for New Sellers with More Than 30 Products Sold in the Last 3 Months (latest date for order_purchase_timestamp is the latest date in the database)
query_new_sellers_final = """
WITH LatestDate AS (
    SELECT MAX(order_purchase_timestamp) AS max_date
    FROM orders
),
ThreeMonthsAgo AS (
    SELECT datetime(MAX(order_purchase_timestamp), '-3 months') AS three_months_ago
    FROM orders
),
SellerFirstOrder AS (
    SELECT 
        i.seller_id,
        MIN(o.order_purchase_timestamp) AS first_order_date
    FROM order_items AS i
    JOIN orders AS o ON i.order_id = o.order_id
    GROUP BY i.seller_id
),
NewSellers AS (
    SELECT 
        sfo.seller_id,
        COUNT(i.order_item_id) AS total_products_sold
    FROM SellerFirstOrder AS sfo
    JOIN order_items AS i ON sfo.seller_id = i.seller_id
    JOIN orders AS o ON i.order_id = o.order_id
    WHERE 
        sfo.first_order_date >= (SELECT three_months_ago FROM ThreeMonthsAgo)
        AND sfo.first_order_date <= (SELECT max_date FROM LatestDate)
    GROUP BY sfo.seller_id
    HAVING total_products_sold > 30
)
SELECT ns.seller_id, ns.total_products_sold
FROM NewSellers AS ns
JOIN sellers AS s ON ns.seller_id = s.seller_id;
"""

# Execute the combined query and load data into a DataFrame
data_new_sellers_final = execute_query(query_new_sellers_final)

# Print the shape of the DataFrame
num_sellers = data_new_sellers_final.shape[0]
print(f"DataFrame: new_sellers_final")
print(f"Number of sellers: {num_sellers}")
print(f"Shape of the resulting DataFrame: {data_new_sellers_final.shape}")

if data_new_sellers_final.empty:
    print(f"No data available. This may be due to the date range issue because the latest date in the database is {today}.")
else:
    print("Query executed successfully.")
    print(data_new_sellers_final)  # Display the DataFrame


DataFrame: new_sellers_final
Number of sellers: 2
Shape of the resulting DataFrame: (2, 2)
Query executed successfully.
                          seller_id  total_products_sold
0  d13e50eaa47b4cbe9eb81465865d8cfc                   69
1  81f89e42267213cb94da7ddc301651da                   52


### 4.Which are the 5 zip codes with more than 30 reviews that have the worst average review scores over the last 12 months considering the date range issue (latest date for review_creation_date as the latest date in the database)    

### Explanation of the Query
1. **Average Review Scores**: The query calculates the average review scores for each zip code based on the reviews received over the last 12 months.
2. **Filtering Criteria**: The query filters the zip codes based on the number of reviews received, selecting only those zip codes with more than 30 reviews.
3. **Common Table Expression (CTE)**: The query uses a Common Table Expression (CTE) named `RecentReviews` to filter the reviews based on the review creation date within the last 12 months.
4. **Result Columns**: The final result includes the `customer_zip_code_prefix` and `average_score` columns for the selected zip codes.
5. **Data Loading**: The query result is loaded into a pandas DataFrame for further analysis and display.
6. **Display**: The first few rows of the resulting DataFrame are displayed to show the 5 zip codes with the worst average review scores over the last 12 months.
7. **Context**: The query considers the date range issue and provides additional information if no data is available due to older dates in the database.

### Justification of Using Review Creation Date instead of Order Date   

1. **Relevance of Reviews**: Reviews are often left after the products are received. Therefore, the review creation date is more directly tied to customer satisfaction and perceived quality of the service or product.
2. **Time Elapsed Since Order**: There can be a significant lag between the order date (order_purchase_timestamp) and the review creation date, especially if delivery takes time or customers delay leaving a review.
3. **Temporal Accuracy**: Using the review creation date provides a more precise and current measure of reviews left within the last 12 months.

From a logical standpoint, the order_purchase_timestamp cannot be used as it does not accurately reflect the timing of customer feedback.

In [21]:
import pandas as pd
import sqlite3
import os
from datetime import datetime

# Get today's date
today = datetime.today().strftime('%Y-%m-%d')

# Relative path to the database
db_path = os.path.join('..', 'data', 'olist.db')

# Function to execute SQL queries and return DataFrames
def execute_query(query):
    with sqlite3.connect(db_path) as conn:
        return pd.read_sql(query, conn)

# SQL query for the 5 zip codes with more than 30 reviews and the worst average review scores over the last 12 months 
# Review_creation_date is latest date in the database (more relevant for review analysis than order_purchase_timestamp)
query_zip_codes_reviews = """
WITH LatestReviewDate AS (
    SELECT MAX(review_creation_date) AS max_review_date
    FROM order_reviews
),
OneYearAgo AS (
    SELECT datetime(MAX(review_creation_date), '-12 months') AS one_year_ago
    FROM order_reviews
),
RecentReviews AS (
    SELECT 
        r.review_id,
        r.review_score,
        c.customer_zip_code_prefix,
        r.review_creation_date
    FROM order_reviews AS r
    JOIN orders AS o ON r.order_id = o.order_id
    JOIN customers AS c ON o.customer_id = c.customer_id
    WHERE 
        r.review_creation_date >= (SELECT one_year_ago FROM OneYearAgo)
        AND r.review_creation_date <= (SELECT max_review_date FROM LatestReviewDate)
),
ZipScores AS (
    SELECT 
        customer_zip_code_prefix,
        AVG(review_score) AS average_score,
        COUNT(review_id) AS review_count
    FROM RecentReviews
    GROUP BY customer_zip_code_prefix
    HAVING review_count > 30
)
SELECT 
    customer_zip_code_prefix,
    average_score
FROM ZipScores
ORDER BY average_score ASC
LIMIT 5;
"""

# Execute the combined query and load data into a DataFrame
data_zip_codes_reviews = execute_query(query_zip_codes_reviews)

# Print the shape of the DataFrame and display the results
print("DataFrame: zip_codes_reviews")
num_zip_codes = data_zip_codes_reviews.shape[0]
print(f"Number of zip codes: {num_zip_codes}")
print(f"Shape of the resulting DataFrame: {data_zip_codes_reviews.shape}")

if data_zip_codes_reviews.empty:
    most_recent_date_query = """
    SELECT MAX(review_creation_date) AS most_recent_review_date
    FROM order_reviews;
    """
    data_recent_review_date = execute_query(most_recent_date_query)
    most_recent_date = data_recent_review_date['most_recent_review_date'].iloc[0]
    most_recent_date = datetime.strptime(most_recent_date, '%Y-%m-%d %H:%M:%S')
    print(f"No data available. This may be due to the date range issue because the most recent review date in the database is {most_recent_date}.")
else:
    print("Query executed successfully.")
    print(data_zip_codes_reviews)  # Display the DataFrame


DataFrame: zip_codes_reviews
Number of zip codes: 5
Shape of the resulting DataFrame: (5, 2)
Query executed successfully.
   customer_zip_code_prefix  average_score
0                     22753       2.867925
1                     22723       3.000000
2                     28893       3.125000
3                     22770       3.184211
4                     13056       3.272727
